In [70]:
import torch
import torch.nn as nn
import torchvision.models as models
import pytorch_lightning as pl
import numpy as np
from torch.utils.data import DataLoader, Dataset
from typing import Any, Callable, Dict, List, Optional, Tuple, cast

In [74]:
import models

In [4]:
root_path = "E:\\subset"
output_path = "C:\\Users\\jeong\\Desktop\\python\\action-recognition-practice\\preprocessed\\"

In [13]:
cfg = np.load(output_path+'cfg.npz', allow_pickle=True)
classes, lookup = cfg['classes'], cfg['lookup']

In [75]:
model = models.R2Plus1DEncoder(num_classes=len(classes))

In [64]:
loaded = np.load(directory_path + str(index) + '.npz')
target = loaded['target']
clip = loaded['clip']

In [66]:
clip.shape

(3, 6, 480, 768)

In [82]:
class TestDataset(Dataset):
    def __init__(self, directory_path: str):
        self.directory_path = directory_path
        self.cfg = np.load(directory_path+'cfg.npz', allow_pickle=True)
        self.classes, lookup = cfg['classes'], cfg['lookup']

    def __len__(self) -> int:
        return len(os.listdir(self.directory_path)) - 1

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        loaded = np.load(directory_path + str(index) + '.npz')
        target = loaded['target'] / 255.0
        clip = loaded['clip']

        return clip, torch.tensor(
            target.item(), dtype=torch.long)  # _onehot(target, self.num_classes)


class TestDataModule(pl.LightningDataModule):
    def __init__(self, directory_path: str, batch_size=1):
        super().__init__()
        self.directory_path = directory_path
        self.batch_size = batch_size

    def setup(self, stage):
        def to_tensor(x):
            x = torch.from_numpy(x)
            return x.float().div(255)

        # transform = transforms.Compose([to_tensor])
        self.dataset = TestDataset(self.directory_path)

    # return the dataloader for each split
    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size)


In [83]:
dm = TestDataModule(output_path, 1)

In [84]:
trainer = pl.Trainer(gpus=1)
trainer.fit(model, dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params
--------------------------------------
0 | model | VideoResNet | 31 M  
Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same